In [70]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import time
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import itertools
from collections import OrderedDict

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf
from fitter import misc as ms

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [71]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)
reload(ms)

<module 'fitter.misc' from '../fitter\misc.pyc'>

### Make fit

In [55]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 0, # max 10
        'exclude' : [] # put LECs here
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'xpt-taylor', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : False,
    
    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400', u'a15m130', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
}

In [56]:
fit_results = OrderedDict()

nnlo_lecs = ['A_a', 'A_k', 'A_p']
for k in range(len(nnlo_lecs)+1):
    for subset in itertools.combinations(nnlo_lecs, k):
        print list(subset)
        p_dict['order']['exclude'] = list(subset)
        
        included_lecs = list(set(nnlo_lecs).difference(subset))

        t0 = time.time()

        # Load data
        data_loader = dl.data_loader()
        fit_data = data_loader.get_fit_data()

        # Get prior
        prior = None
        if p_dict['use_prior']:
            prior = data_loader.get_prior(p_dict['fit_type'])

        # Make bootstrapper
        bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                                       fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

        if p_dict['make_plots']:
            data_loader.save_plots(
                bootstrapper.make_plots(
                    show_error_ellipses=p_dict['show_error_ellipses'], 
                    show_bootstrap_histograms=p_dict['show_bs_histograms']),
                output_filename="fits_by_lec/"+bootstrapper.fit_type+'_'+bootstrapper.order['fit']+'_sans_'+str(p_dict['order']['exclude'])
            )

        # Save results for comparison
        fit_results[str(included_lecs)] = bootstrapper.get_fit_info()

        t1 = time.time()

        print "\nTotal time (s): ", t1 - t0, "\n"

[]
Making fits...
100% complete Time (s):  0.302000045776
Compiling results...

Total time (s):  0.614000082016 

['A_a']
Making fits...
100% complete Time (s):  0.332000017166
Compiling results...

Total time (s):  0.640999794006 

['A_k']
Making fits...
100% complete Time (s):  0.355999946594
Compiling results...

Total time (s):  0.668999910355 

['A_p']
Making fits...
100% complete Time (s):  0.248999834061
Compiling results...

Total time (s):  0.56200003624 

['A_a', 'A_k']
Making fits...
100% complete Time (s):  0.252999782562
Compiling results...

Total time (s):  0.545000076294 

['A_a', 'A_p']
Making fits...
100% complete Time (s):  0.232000112534
Compiling results...

Total time (s):  0.528000116348 

['A_k', 'A_p']
Making fits...
100% complete Time (s):  0.191999912262
Compiling results...

Total time (s):  0.500999927521 

['A_a', 'A_k', 'A_p']
Making fits...
100% complete Time (s):  0.226999998093
Compiling results...

Total time (s):  0.514999866486 



In [67]:
xlabel = '$F^\pm_K/F^\pm_\pi$'
title = bootstrapper.fit_type+", chained: "+str(bootstrapper.chain_fits)+",\nspecified NNLO LECS only"
filename = "exclude_lecs/"+bootstrapper.fit_type+"__chained-"+str(bootstrapper.chain_fits)+"__exclude-n2lo"
fig = ms.plot_comparison('fit', fit_results, title=title, xlabel=xlabel, show_model_avg=True)
data_loader.save_plots(fig, output_filename=filename)

Done.
